# Підготовка даних до аналізу

Перед будь-яким аналізом дані необхідно обробити. Треба встановити відповідні типи даних, опрацювати пропуски в даних та виправити можливі помилки

Для початку роботи треба завантажити необхідні бібліотеки - pandas та re

In [52]:
import pandas as pd
import re

Наступний код завантажить робочий датасет Laptop_prices.csv

In [53]:
from google.colab import drive
from google.colab import files

drive.mount('/content/drive')
filename = "/content/drive/My Drive/Laptop_prices.csv"

df = pd.read_csv(filename)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Виведемо перші 5 записів датафрейму. Одразу можна помітити відсутні начення (NaN)

In [54]:
df.head(5)

,Unnamed: 0,Name,Brand,Price,Rating,Processor_brand,Processor_name,Processor_variant,Processor_gen,Core_per_processor,...,Graphics_name,Graphics_brand,Graphics_GB,Graphics_integreted,Display_size_inches,Horizontal_pixel,Vertical_pixel,ppi,Touch_screen,Operating_system
0,0,HP Victus 15-fb0157AX Gaming Laptop (AMD Ryzen...,HP,50399,4.30,AMD,AMD Ryzen 5,5600H,5.0,6.0,...,AMD Radeon RX 6500M,AMD,4.0,False,15.6,1920,1080,141.21,True,Windows 11 OS
1,1,Lenovo V15 G4 ‎82YU00W7IN Laptop (AMD Ryzen 3 ...,Lenovo,26690,4.45,AMD,AMD Ryzen 3,7320U,7.0,4.0,...,AMD Radeon Graphics,AMD,NaN,False,15.6,1920,1080,141.21,False,Windows 11 OS
2,2,HP 15s-fq5007TU Laptop (12th Gen Core i3/ 8GB/...,HP,37012,4.65,Intel,Intel Core i3,1215U,12.0,6.0,...,Intel UHD Graphics,Intel,NaN,False,15.6,1920,1080,141.21,False,Windows 11 OS
3,3,Samsung Galaxy Book2 Pro 13 Laptop (12th Gen C...,Samsung,69990,4.75,Intel,Intel Core i5,1240P,12.0,12.0,...,Intel Iris Xe Graphics,Intel,NaN,False,13.3,1080,1920,165.63,False,Windows 11 OS
4,4,Tecno Megabook T1 Laptop (11th Gen Core i3/ 8G...,Tecno,23990,4.25,Intel,Intel Core i3,1115G4,11.0,2.0,...,Intel UHD Graphics,Intel,NaN,False,15.6,1920,1080,141.21,False,Windows 11 OS


## Заповнення порожніх значень

Для початку підготовки даних до аналізу дослідимо структуру

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              1020 non-null   int64  
 1   Name                    1020 non-null   object 
 2   Brand                   1020 non-null   object 
 3   Price                   1020 non-null   int64  
 4   Rating                  1020 non-null   float64
 5   Processor_brand         1020 non-null   object 
 6   Processor_name          1020 non-null   object 
 7   Processor_variant       996 non-null    object 
 8   Processor_gen           891 non-null    float64
 9   Core_per_processor      1008 non-null   float64
 10  Total_processor         573 non-null    float64
 11  Execution_units         573 non-null    float64
 12  Low_Power_Cores         1020 non-null   float64
 13  Energy_Efficient_Units  1020 non-null   int64  
 14  Threads                 972 non-null    

В датасеті багато пропущених значень. Подивимося на них детальніше:

*   24 пропущених значень Processor_variant (моделі процесорів)
*   125 пропущених значень Processor_gen (покоління процесорів)
*   12 пропущених значень Core_per_processor (кількість ядер)
*   По 447 пропущених значень Total_processor та Execution_units
*   38 пропущених значень Threads (кількість потоків)
*   22 пропущених значення RAM_type (тип ОЗП)
*   2 пропущені значення Graphics_name (назви відеоядер)
*   2 пропущені значення Graphics_brand (назви виробників відеодер)
*   652 пропущенних значення Graphics_GB (об'єм відеопам'яті)
*   2 пропущені значення Graphics_integreted (характеристика встроєності відеоядра)

Спробуємо ці дані виправити

Окрім пропущених значень, помітні такі проблеми:

*   Колонка з унікальним id не має назви
*   Колонка Graphics_integreted названа неправильно

Виправимо ці помилки.

In [56]:
df = df.rename(columns={df.columns[0]: 'id'})
df = df.rename(columns={'Graphics_integreted': 'Graphics_integrated'})

Для початку випавлення помилок розберемось із Processor_variant.

В цій колонці містяться дані про конкретну модель процесора. Подивимось на пропущені значення:

In [57]:
filtered_df = df[df['Processor_variant'].isna()]
result_df = filtered_df[['id', 'Name', 'Processor_brand', 'Processor_name']]
result_df

,id,Name,Processor_brand,Processor_name
13,13,Apple MacBook Air 2020 MGND3HN Laptop (Apple M...,Apple,Apple M1
17,17,Apple MacBook Air 2024 Laptop (Apple M3/ 8GB/ ...,Apple,Apple M3
23,23,Apple MacBook Air 2022 Laptop (Apple M2/ 8GB/ ...,Apple,Apple M2
73,73,Apple MacBook Air 2024 MRYN3HN/A Laptop (Apple...,Apple,Apple M3
83,83,Apple MacBook Pro 16 2023 Laptop (Apple M3 Max...,Apple,Apple M3 Max
100,100,Apple MacBook Air 15 2024 MRYR3HN/A Laptop (Ap...,Apple,Apple M3
116,116,Apple MacBook Pro 14 2023 Laptop (Apple M3 Pro...,Apple,Apple M3 Pro
138,138,Apple MacBook Air 2022 Laptop (Apple M2/ 8GB/ ...,Apple,Apple M2 Apple M2 Chip
154,154,Lenovo Ideapad Slim 3 Chrome 14M868 82XJ002RHA...,MediaTek,MediaTek Kompanio
163,163,Apple MacBook Pro 14 2023 Laptop (Apple M3/ 8G...,Apple,Apple M3


Можна помітити наступну закономірність: значення пропущені виключно для процесорів виробників Apple та MediaTek.

Виправимо ці дані.

Для техніки виробника Apple модель процесора візьмемо з його назви

In [58]:
for index, row in df.iterrows():
  if row['Processor_brand'] == 'Apple' and pd.isna(row['Processor_variant']):
    parts = row['Processor_name'].split('Apple')
    if len(parts) > 2:
      new_variant = parts[2].replace('Chip', '').strip()
    else:
      new_variant = parts[1].replace('Chip', '').strip()
    df.at[index, 'Processor_variant'] = new_variant

Для техніки з процесорами від виробника MediaTek назву процесору візьмемо з назви ноутбука

In [59]:
for index, row in df.iterrows():
  if 'MediaTek' in row['Processor_brand'] and pd.isna(row['Processor_variant']):
    name_parts = row['Name'].split()
    for i in range(len(name_parts)):
      if 'MediaTek' in name_parts[i]:
        if i + 1 < len(name_parts):
          next_part = name_parts[i+1].strip('/')
          if not any(char.isdigit() for char in next_part):
            if i + 2 < len(name_parts):
              df.at[index, 'Processor_variant'] = name_parts[i + 2].strip('/')
          else:
            df.at[index, 'Processor_variant'] = next_part
            break

Перевіримо правильність виконаних дій і виведемо моделі ноутбуків із процесорами від Apple та MediaTek.

Дані про модель процесора успішно заповнено.

In [60]:
filtered_df = df[(df['Processor_brand'] == 'Apple') | (df['Processor_brand'] == 'MediaTek')]
result_df = filtered_df[['id', 'Name', 'Processor_brand', 'Processor_name', 'Processor_variant']]
result_df

,id,Name,Processor_brand,Processor_name,Processor_variant
13,13,Apple MacBook Air 2020 MGND3HN Laptop (Apple M...,Apple,Apple M1,M1
17,17,Apple MacBook Air 2024 Laptop (Apple M3/ 8GB/ ...,Apple,Apple M3,M3
23,23,Apple MacBook Air 2022 Laptop (Apple M2/ 8GB/ ...,Apple,Apple M2,M2
73,73,Apple MacBook Air 2024 MRYN3HN/A Laptop (Apple...,Apple,Apple M3,M3
83,83,Apple MacBook Pro 16 2023 Laptop (Apple M3 Max...,Apple,Apple M3 Max,M3 Max
100,100,Apple MacBook Air 15 2024 MRYR3HN/A Laptop (Ap...,Apple,Apple M3,M3
116,116,Apple MacBook Pro 14 2023 Laptop (Apple M3 Pro...,Apple,Apple M3 Pro,M3 Pro
138,138,Apple MacBook Air 2022 Laptop (Apple M2/ 8GB/ ...,Apple,Apple M2 Apple M2 Chip,M2
154,154,Lenovo Ideapad Slim 3 Chrome 14M868 82XJ002RHA...,MediaTek,MediaTek Kompanio,520
163,163,Apple MacBook Pro 14 2023 Laptop (Apple M3/ 8G...,Apple,Apple M3,M3


Далі заповнимо пропущені значення для покоління процесорів за такими правилами:

*   Для процесорів Apple покоління візьмемо з його назви
*   Для процесорів інших виробників покоління заповнимо модою
*   Для незаповнених значень встановимо 1 покоління, адже номер покоління таких процесорів у датасеті не зустрічаються


In [61]:
mode_per_brand = df.groupby('Processor_brand')['Processor_gen'].agg(lambda x: pd.Series.mode(x).iloc[0] if not x.mode().empty else None)
for brand, mode in mode_per_brand.items():
  if mode is not None:
    df.loc[(df['Processor_brand'] == brand) & (df['Processor_gen'].isna()), 'Processor_gen'] = mode

for index, row in df[df['Processor_brand'] == 'Apple'].iterrows():
  match = re.search(r'M(\d+)', row['Processor_name'])
  if match:
    df.at[index, 'Processor_gen'] = int(match.group(1))

df.loc[df['Processor_gen'].isna(), 'Processor_gen'] = 1

Також модою заповнимо і пропуски кількості ядер процесорів

In [62]:
mode_per_var = df.groupby('Processor_variant')['Core_per_processor'].agg(lambda x: pd.Series.mode(x).iloc[0] if not x.mode().empty else None)
for brand, mode in mode_per_var.items():
  if mode is not None:
    df.loc[(df['Processor_variant'] == brand) & (df['Core_per_processor'].isna()), 'Core_per_processor'] = mode

overall_mode = df['Core_per_processor'].mode().iloc[0] if not df['Core_per_processor'].mode().empty else None
if overall_mode is not None:
    df['Core_per_processor'].fillna(overall_mode, inplace=True)

Колонки Total_processor та Execution_units видалимо, адже майже половина даних відсутня і спроба імітувати їх себе не виправдає

In [63]:
df = df.drop(columns=['Total_processor', 'Execution_units'])

Кількість потоків можна заповнити у декілька етапів:

*   для заповнених значень кількості потоків, слід знайти моду за кількістю ядер процесора. Цим значенням слід заповнити пропуски
*   процесори Apple не використовують технологію Multithreading, тобто не паралелять кожне ядро на декілька потоків. В такому випадку кількість потоків дорівнює кількості ядер
*   всі інші процесори зазвичай використовують технологію Multithreading, тоді кількість ядер слід подвоїти і таким чином знайти кількість потоків для не заповнених на минулих етапах даних



In [64]:
mode_per_brand = df.groupby('Core_per_processor')['Threads'].agg(lambda x: pd.Series.mode(x).iloc[0] if not x.mode().empty else None)
for brand, mode in mode_per_brand.items():
  if mode is not None:
    df.loc[(df['Core_per_processor'] == brand) & (df['Threads'].isna()), 'Threads'] = mode

for index, row in df.iterrows():
  if row['Processor_brand'] == 'Apple' and pd.isna(row['Threads']):
    df.at[index, 'Threads'] = df.at[index, 'Core_per_processor']
  elif pd.isna(row['Threads']):
    df.at[index, 'Threads'] = df.at[index, 'Core_per_processor'] + df.at[index, 'Core_per_processor']

Медіаною заповнюємо і тип ОЗП. Для техніки Apple присвоюємо власний тип "Unified"

In [65]:
for index, row in df.iterrows():
  if row['Processor_brand'] == 'Apple' and pd.isna(row['RAM_type']):
    df.at[index, 'RAM_type'] = 'Unified'

mode_per_GB = df.groupby('RAM_GB')['RAM_type'].agg(lambda x: pd.Series.mode(x).iloc[0] if not x.mode().empty else None)
for ram, mode in mode_per_GB.items():
  if mode is not None:
    df.loc[(df['RAM_GB'] == ram) & (df['RAM_type'].isna()), 'RAM_type'] = mode

Для знаходження характеристики відеоядра теж використаємо моду

In [66]:
overall_mode_name = df['Graphics_name'].mode().iloc[0] if not df['Graphics_name'].mode().empty else None

if overall_mode_name is not None:
  df['Graphics_name'].fillna(overall_mode_name, inplace=True)
  filtered_df = df[df['Graphics_name'] == overall_mode_name]
  overall_mode_brand = filtered_df['Graphics_brand'].mode().iloc[0] if not filtered_df['Graphics_brand'].mode().empty else None
  if overall_mode_brand is not None:
      df.loc[df['Graphics_name'] == overall_mode_name, 'Graphics_brand'] = df.loc[df['Graphics_name'] == overall_mode_name, 'Graphics_brand'].fillna(overall_mode_brand)
  overall_mode_integrated = filtered_df['Graphics_integrated'].mode().iloc[0] if not filtered_df['Graphics_integrated'].mode().empty else None
  if overall_mode_integrated is not None:
      df.loc[df['Graphics_name'] == overall_mode_name, 'Graphics_integrated'] = df.loc[df['Graphics_name'] == overall_mode_name, 'Graphics_integrated'].fillna(overall_mode_integrated)

Більшість записів Graphics_GB (об'єм пам'яті відеоядра) відсутня. Причною цього є той факт, що більшість відеоядер використовують не власну відеопам'ять, а ОЗП. Тому для пропущених значень вказуємо 0

In [67]:
for index, row in df.iterrows():
  if pd.isna(row['Graphics_GB']):
    df.at[index, 'Graphics_GB'] = 0

Перевіряємо заповненість даних. Всі дані успішно заповнено

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      1020 non-null   int64  
 1   Name                    1020 non-null   object 
 2   Brand                   1020 non-null   object 
 3   Price                   1020 non-null   int64  
 4   Rating                  1020 non-null   float64
 5   Processor_brand         1020 non-null   object 
 6   Processor_name          1020 non-null   object 
 7   Processor_variant       1020 non-null   object 
 8   Processor_gen           1020 non-null   float64
 9   Core_per_processor      1020 non-null   float64
 10  Low_Power_Cores         1020 non-null   float64
 11  Energy_Efficient_Units  1020 non-null   int64  
 12  Threads                 1020 non-null   float64
 13  RAM_GB                  1020 non-null   int64  
 14  RAM_type                1020 non-null   

## Виправлення помилок

Для кращого і точнішого аналізу слід виправити всі помилки у даних. Почнемо з виправлення помилок для числових значень

Проаналізувавши типи даних колонок датафрейму, виправимо деякі неточності:

In [69]:
df['Threads'] = df['Threads'].astype('int64')
df['Graphics_GB'] = df['Graphics_GB'].astype('int64')
df['Processor_gen'] = df['Processor_gen'].astype('int64')
df['Core_per_processor'] = df['Core_per_processor'].astype('int64')
df['Graphics_integrated'] = df['Graphics_integrated'].astype('bool')

Опишемо числові дані за допомогою .describe()

In [70]:
df.describe()

,id,Price,Rating,Processor_gen,Core_per_processor,Low_Power_Cores,Energy_Efficient_Units,Threads,RAM_GB,Storage_capacity_GB,Graphics_GB,Display_size_inches,Horizontal_pixel,Vertical_pixel,ppi
count,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000,1020.000000
mean,509.500000,82063.474510,4.373676,10.442157,8.577451,0.086275,0.043137,12.968627,13.992157,627.733333,2.143137,15.163775,2035.512745,1214.019608,157.178265
std,294.592939,66502.150607,0.233295,3.213317,4.386714,0.406531,0.203266,5.705652,7.189564,316.911679,3.272256,1.001537,409.209289,306.863086,33.585713
min,0.000000,8000.000000,3.950000,1.000000,2.000000,0.000000,0.000000,2.000000,2.000000,32.000000,0.000000,11.600000,1080.000000,768.000000,100.450000
25%,254.750000,43990.000000,4.200000,7.000000,6.000000,0.000000,0.000000,8.000000,8.000000,512.000000,0.000000,14.000000,1920.000000,1080.000000,141.210000
50%,509.500000,63689.500000,4.350000,12.000000,8.000000,0.000000,0.000000,12.000000,16.000000,512.000000,0.000000,15.600000,1920.000000,1080.000000,141.210000
75%,764.250000,94990.000000,4.550000,13.000000,10.000000,0.000000,0.000000,16.000000,16.000000,512.000000,4.000000,15.600000,1920.000000,1200.000000,161.730000
max,1019.000000,599990.000000,4.750000,14.000000,24.000000,2.000000,1.000000,32.000000,64.000000,4000.000000,16.000000,18.000000,3840.000000,2560.000000,337.930000


Бачимо наступне: середні значення Low_Power_Cores та Energy_Efficient_Units майже = 0. Ці дані явно порожні, їх варто видалити

In [71]:
df = df.drop(columns=['Low_Power_Cores', 'Energy_Efficient_Units'])

Всі характеристики, що мають абсолютне значення, додатні.

Помилок не знайдено, хоча значення 1080 для Horizontal_pixel явно не є типовим. Перевіримо цю здогадку

In [72]:
filtered_df = df[df['Horizontal_pixel'] == 1080]
filtered_df[['Vertical_pixel', 'Horizontal_pixel']].head(10)

,Vertical_pixel,Horizontal_pixel
3,1920,1080
10,1920,1080
42,1920,1080
412,1920,1080
560,1920,1080
689,1920,1080
813,1920,1080


Знайдено наступну помилку: у деяких ноутбуків переплутані параметри розширеня екрану. Горизонтальна кількість пікселів має бути вищою за вертикальну, адже всі ноутбуки виготовляються горизонтальними, а не вертикальними. Виправимо це

In [73]:
for index, row in df.iterrows():
  if row['Horizontal_pixel'] < row['Vertical_pixel']:
    df.at[index, 'Horizontal_pixel'], df.at[index, 'Vertical_pixel'] = row['Vertical_pixel'], row['Horizontal_pixel']

Перевіримо успішність виправлення. Дані виправлено успішно

In [74]:
for index, row in df.iterrows():
  if row['Horizontal_pixel'] < row['Vertical_pixel']:
    print(df.loc[index])

Перевіримо категоріальні значення на правильність написання.

Перевірку почнемо з назви брендів процесору. Помилок не знайдено

In [75]:
sorted(df['Processor_brand'].unique())

['AMD', 'Apple', 'HiSilicon', 'Intel', 'MediaTek', 'Microsoft', 'Qualcomm']

Також перевіримо правильність написання назви лінійки процесорів

In [76]:
sorted(df['Processor_name'].unique())

['AMD Athlon',
 'AMD Athlon Pro',
 'AMD Athlon Silver',
 'AMD Ryzen 3',
 'AMD Ryzen 5',
 'AMD Ryzen 7',
 'AMD Ryzen 9',
 'Apple M1',
 'Apple M2',
 'Apple M2 Apple M2 Chip',
 'Apple M3',
 'Apple M3 Max',
 'Apple M3 Pro',
 'HiSilicon  Kirin 9006C 9006C',
 'Intel  ',
 'Intel Atom Quad',
 'Intel Celeron ',
 'Intel Celeron Dual',
 'Intel Core 3',
 'Intel Core 5',
 'Intel Core 7',
 'Intel Core I3-1115G4',
 'Intel Core Ultra',
 'Intel Core i3',
 'Intel Core i5',
 'Intel Core i7',
 'Intel Core i9',
 'Intel Pentium ',
 'Intel Pentium Gold',
 'Intel Pentium Silver',
 'MediaTek',
 'MediaTek Kompanio',
 'Mediatek',
 'Microsoft SQ1 SQ1',
 'Qualcomm X Elite',
 'eration Intel Core']

Знайдено наступні помилки:

*   Apple M3 Max та Apple M3 Pro - вказана конкретна модель процесора, а не лінійка. Слід виправити
*   Apple M2 Apple M2 Chip - має бути просто Apple M2
*   "Intel  " та "Intel Pentium " - мають зайві пробіли в кінці
*   Intel Core I3-1115G4 - має бути Intel Core i3, а в колонці Processor_variant має залишитись 1115G4
*   HiSilicon  Kirin 9006C 9006C - має бути просто HiSilicon Kirin, а в колонці Processor_variant слід залишити 9006C
*   Intel Core 3, 5, 7 - має бути Intel Core i3, i5, i7
*   MediaTek та Mediatek варто замінити на MediaTek Kompanio
*   Microsoft SQ1 SQ1 слід замінти на Microsoft, а в колонці Processor_variant слід залишити SQ1
*   Qualcomm X Elite слід замінти на Qualcomm X, а в колонці Processor_variant слід залишити Elite
*   з "eration Intel Core" слід розібратися детальніше окремо

In [77]:
for index, row in df.iterrows():
  processor_name = row['Processor_name'].replace("  ", " ").strip()
  if 'Apple M3' in processor_name:
      processor_name = 'Apple M3'
  elif 'Apple M2 Apple M2 Chip' in processor_name:
      processor_name = 'Apple M2'
      df.at[index, 'Processor_variant'] = 'M2'
  elif processor_name.startswith("Intel ") and processor_name.endswith(" "):
      processor_name = processor_name.strip()
  elif 'Intel Core I3-1115G4' in processor_name:
      processor_name = 'Intel Core i3'
      df.at[index, 'Processor_variant'] = '1115G4'
  elif 'HiSilicon Kirin 9006C 9006C' in processor_name:
      processor_name = 'HiSilicon Kirin'
      df.at[index, 'Processor_variant'] = '9006C'
  elif 'Intel Core 3' in processor_name:
      processor_name = processor_name.replace('3', 'i3')
  elif 'Intel Core 5' in processor_name:
      processor_name = processor_name.replace('5', 'i5')
  elif 'Intel Core 7' in processor_name:
      processor_name = processor_name.replace('7', 'i7')
  elif 'MediaTek' in processor_name or 'Mediatek' in processor_name:
      processor_name = 'MediaTek Kompanio'
  elif 'Microsoft SQ1 SQ1' in processor_name:
      processor_name = 'Microsoft'
      df.at[index, 'Processor_variant'] = 'SQ1'
  elif 'Qualcomm X Elite' in processor_name:
      processor_name = 'Qualcomm X'
      df.at[index, 'Processor_variant'] = 'Elite'

  df.at[index, 'Processor_name'] = processor_name

З "eration Intel Core" слід розібратися окремо. Помилка нетипова, подивимося на модель детальніше:

In [78]:
filtered_df = df[df['Processor_name'] == 'eration Intel Core']
filtered_df[['Name', 'Processor_name', 'Processor_variant']]

,Name,Processor_name,Processor_variant
140,Samsung Galaxy Book2 15 Laptop (12th Gen Core ...,eration Intel Core,L3…


Всі дані про процесор помилкові. Їх можна відновити за назвою ноутбуку

In [79]:
for word in filtered_df['Name']:
  print(word)

Samsung Galaxy Book2 15 Laptop (12th Gen Core i5/ 8GB/ 512GB SSD/ Win11)


Цей процесор - це Intel Core i5-1235U

In [80]:
index = df[df['Processor_name'] == 'eration Intel Core'].index

if not index.empty:
  df.at[index[0], 'Processor_name'] = 'Intel Core i5'
  df.at[index[0], 'Processor_variant'] = '1235U'
  df.at[index[0], 'Processor_gen'] = 12
  df.at[index[0], 'Core_per_processor'] = 10
  df.at[index[0], 'Threads'] = 12

Перевіримо успішність виправлення помилок написання лінійок процесорів. Дані виправлено успішно

In [81]:
sorted(df['Processor_name'].unique())

['AMD Athlon',
 'AMD Athlon Pro',
 'AMD Athlon Silver',
 'AMD Ryzen 3',
 'AMD Ryzen 5',
 'AMD Ryzen 7',
 'AMD Ryzen 9',
 'Apple M1',
 'Apple M2',
 'Apple M3',
 'HiSilicon Kirin',
 'Intel',
 'Intel Atom Quad',
 'Intel Celeron',
 'Intel Celeron Dual',
 'Intel Core Ultra',
 'Intel Core i3',
 'Intel Core i5',
 'Intel Core i7',
 'Intel Core i9',
 'Intel Pentium',
 'Intel Pentium Gold',
 'Intel Pentium Silver',
 'MediaTek Kompanio',
 'Microsoft',
 'Qualcomm X']

Перейдемо до Processor_variant і пошукаємо помилки тут

In [82]:
sorted(df['Processor_variant'].unique())

['1005G1',
 '100U',
 '10210U',
 '10210Y',
 '1035G1',
 '1035G4',
 '10510U',
 '10750H',
 '1115G4',
 '1125G4',
 '11260H',
 '11300H',
 '11320H',
 '1135G7',
 '11400H',
 '1155G7',
 '1165G7',
 '11800H',
 '1195G7',
 '120U',
 '1215U',
 '1220P',
 '1230U',
 '1235U',
 '1240P',
 '12450H',
 '12450HX',
 '12500H',
 '1250U',
 '1255U',
 '125H',
 '125U',
 '1260P',
 '12650H',
 '12700H',
 '12900H',
 '12900HX',
 '1305U',
 '1315U',
 '1334U',
 '1335U',
 '1340P',
 '1340p',
 '13420H',
 '13450HX',
 '13500H',
 '13500HX',
 '1355U',
 '1360P',
 '13620H',
 '13650HX',
 '1365U',
 '13700H',
 '13700HX',
 '13900H',
 '13900HK',
 '13900HX',
 '13950HX',
 '13980HX',
 '14700HX',
 '14900HX',
 '150U',
 '155H',
 '155U',
 '185H',
 '3045B',
 '3050U',
 '3250U',
 '3500U',
 '500',
 '520',
 '5300U',
 '5425U',
 '5500H',
 '5500U',
 '5500u',
 '5600H',
 '5600HS',
 '5625U',
 '5700U',
 '5800H',
 '5800HS',
 '5800U',
 '5825U',
 '6600H',
 '6800H',
 '6800HS',
 '6900HS\u200b',
 '7120U',
 '7320U',
 '7330U',
 '7505',
 '7520U',
 '7530U',
 '7535HS',


Помітні наступні помилки:

*   Деякі суфікси назв у нижньому регістрі, деякі - у верхньому
*   Символ \u200b слід видалити
*   пропущені i5, 7, 9 слід замінити на найпопулярнішу модель серед лінійки i5, 7 і 9 відповідно

Виправимо дані


In [83]:
df['Processor_variant'] = df['Processor_variant'].replace('6900HS\u200b', '6900HS', regex = True)

mode_value = df.loc[df['Processor_name'].str.contains('Intel Core i5', na = False), 'Processor_variant'].mode()[0]
if pd.notna(mode_value):
  df['Processor_variant'] = df['Processor_variant'].replace('i5', mode_value, regex = True)

  mode_value = df.loc[df['Processor_name'].str.contains('Intel Core i7', na = False), 'Processor_variant'].mode()[0]
if pd.notna(mode_value):
  df['Processor_variant'] = df['Processor_variant'].replace('i7', mode_value, regex = True)

  mode_value = df.loc[df['Processor_name'].str.contains('Intel Core i9', na = False), 'Processor_variant'].mode()[0]
if pd.notna(mode_value):
  df['Processor_variant'] = df['Processor_variant'].replace('i9', mode_value, regex = True)

df['Processor_variant'] = df['Processor_variant'].str.upper()

Перевіримо правильність виправлення

In [84]:
sorted(df['Processor_variant'].unique())

['1005G1',
 '100U',
 '10210U',
 '10210Y',
 '1035G1',
 '1035G4',
 '10510U',
 '10750H',
 '1115G4',
 '1125G4',
 '11260H',
 '11300H',
 '11320H',
 '1135G7',
 '11400H',
 '1155G7',
 '1165G7',
 '11800H',
 '1195G7',
 '120U',
 '1215U',
 '1220P',
 '1230U',
 '1235U',
 '1240P',
 '12450H',
 '12450HX',
 '12500H',
 '1250U',
 '1255U',
 '125H',
 '125U',
 '1260P',
 '12650H',
 '12700H',
 '12900H',
 '12900HX',
 '1305U',
 '1315U',
 '1334U',
 '1335U',
 '1340P',
 '13420H',
 '13450HX',
 '13500H',
 '13500HX',
 '1355U',
 '1360P',
 '13620H',
 '13650HX',
 '1365U',
 '13700H',
 '13700HX',
 '13900H',
 '13900HK',
 '13900HX',
 '13950HX',
 '13980HX',
 '14700HX',
 '14900HX',
 '150U',
 '155H',
 '155U',
 '185H',
 '3045B',
 '3050U',
 '3250U',
 '3500U',
 '500',
 '520',
 '5300U',
 '5425U',
 '5500H',
 '5500U',
 '5600H',
 '5600HS',
 '5625U',
 '5700U',
 '5800H',
 '5800HS',
 '5800U',
 '5825U',
 '6600H',
 '6800H',
 '6800HS',
 '6900HS',
 '7120U',
 '7320U',
 '7330U',
 '7505',
 '7520U',
 '7530U',
 '7535HS',
 '7730U',
 '7735HS',
 '773

Наступним кроком перевіримо правильність написання типів ОЗП

In [85]:
sorted(df['RAM_type'].unique())

['DDR3',
 'DDR4',
 'DDR5',
 'DDR6',
 'LPDDR3',
 'LPDDR4',
 'LPDDR4X',
 'LPDDR5',
 'LPDDR5X',
 'LPDDRX4',
 'PDDR5X',
 'Unified']

Помітні наступні помилки: PDDR5X слід виправити на LPDDR5X (префікс LP означає Low-Power), а LPDDRX4 - на LPDDR4X

In [86]:
df['RAM_type'] = df['RAM_type'].replace('LPDDR5X', 'PDDR5X', regex = True)
df['RAM_type'] = df['RAM_type'].replace('PDDR5X', 'LPDDR5X', regex = True)
df['RAM_type'] = df['RAM_type'].replace('LPDDRX4', 'LPDDR4X', regex = True)

Перевіримо Storage_type на правильність написання типу накопичувача

In [87]:
sorted(df['Storage_type'].unique())

[' Hard Disk', ' NVMe SSD', ' SSD', 'Hard Disk & SSD']

Помилок нема, хоча варто видалити зайві пробіли, а Hard Disk замінити на більш прийнятне HDD

In [88]:
df['Storage_type'] = df['Storage_type'].str.strip()
df['Storage_type'] = df['Storage_type'].replace('Hard Disk', 'HDD', regex = True)

Перевіримо колонку Graphics_brand

In [89]:
sorted(df['Graphics_brand'].unique())

['AMD', 'ARM', 'Adreno', 'Apple', 'Intel', 'NVIDIA', 'Radeon']

В цій колонці Radeon слід замінити на AMD

In [90]:
df['Graphics_brand'] = df['Graphics_brand'].replace('Radeon', 'AMD', regex = True)

Перевіримо колонку Graphics_name

In [91]:
sorted(df['Graphics_name'].unique())

['10 Core GPU',
 '10-Core GPU',
 '14 Core GPU',
 '18 Core GPU',
 '30 Core GPU',
 '40 Core GPU',
 '8-Core GPU',
 'AMD Graphics',
 'AMD Integrated',
 'AMD Integrated SoC',
 'AMD Radeon',
 'AMD Radeon  RX 7600S',
 'AMD Radeon 610M',
 'AMD Radeon 610M Graphics',
 'AMD Radeon 680M',
 'AMD Radeon 7 Graphics',
 'AMD Radeon 780M Graphics',
 'AMD Radeon AMD',
 'AMD Radeon AMD Radeon RX 6500M',
 'AMD Radeon Graphics',
 'AMD Radeon Integrated Graphics',
 'AMD Radeon RX 5600M',
 'AMD Radeon RX 6500M',
 'AMD Radeon RX 6500M Graphics',
 'AMD Radeon RX6550M',
 'AMD Radeon Radeon',
 'AMD Radeon Radeon Graphics',
 'AMD Radeon Vega 7',
 'AMD Radeon Vega 7 Graphics',
 'AMD Radeon Vega 8',
 'AMD Radeon Vega 8 Graphics',
 'AMD Raedon Graphics',
 'AMD Vega',
 'ARM Mali G72',
 'Adreno 685',
 'Apple 10 Core GPU',
 'Apple 8 Core GPU',
 'Apple M1 Integrated Graphics',
 'Arm Mali G52 MC2 2EE',
 'Arm Mali G52 MC2 2EE Graphics',
 'Arm Mali-G72 MP3 Graphics',
 'Geforce MX130',
 'Integrated AMD Radeon Graphics',
 'I

Тут можна побачити багато помилок. Їх треба виправити, адже графічне ядро - одна  найголовніших компонент ноутбука.

Помилки наступні:

*   наявні зайві символи: тире, пробіли, ®, \u200e
*   характеристики " Graphics" та "Integrated" зайві
*   є проблеми із написанням назв виробників (напрклад, INTEL, GEFORCE тощо)
*   в деяких характеристиках відсутні пробіли (наприклад, Geforce GTX1650)
*   є дублікати характеристик або ці характеристики пропущені (AMD Radeon, наприклад)
*   всі записи, що містять назву відеядра Intel Iris слід уніфікувати до Intel Iris Xe
*   не вказано виробника відеядер Apple


In [92]:
for index, row in df.iterrows():
  name = row['Graphics_name']

  name = name.replace("-", " ")
  name = name.replace("®", " ")
  name = name.replace("\u200e", "")

  name = name.replace("INTEL", "")
  name = name.replace(" Graphics", "")
  name = name.replace("Integrated", " ")
  name = name.replace("Raedon", "Radeon")
  name = name.replace("Nvidia", "NVIDIA")
  name = name.replace("3050 GPU", "3050")
  name = name.replace("Geforce", "GeForce")
  name = name.replace("GEFORCE", "GeForce")
  name = name.replace("GTX 3050", "RTX 3050")
  name = name.replace("GTX 2050", "RTX 2050")
  name = name.replace("UHD 620", "Intel UHD 620")
  name = name.replace("Intel HD", "Intel UHD 620")

  name = re.sub(r"\s+", " ", name)
  name = re.sub(r"RX\s+", "RX", name)
  name = re.sub(r"RTX(?! \w)", r"RTX ", name)
  name = re.sub(r"GTX(?! \w)", r"GTX ", name)
  name = re.sub(r"(?<!\s)(Ti)", r" \1", name)
  name = re.sub(r"(?<!NVIDIA\s)(GeForce)", r"NVIDIA \1", name)
  name = re.sub(r"(?<!AMD\s)(Radeon)", r"AMD \1", name)
  name = re.sub(r'\b(AMD Radeon)\b(?:\s+\1\b)+', r'\1', name)
  name = re.sub(r'\b(NVIDIA)\b.*\b\1\b', r'\1', name)
  name = re.sub(r'\b(Radeon)\b.*\b\1\b', r'\1', name)
  name = re.sub(r'\b(Intel)\b.*\b\1\b', r'\1', name)
  name = re.sub(r'\b(AMD)\b.*\b\1\b', r'\1', name)
  name = re.sub(r'\b(RTX)\b.*\b\1\b', r'\1', name)

  if 'GeForce' not in name:
    name = re.sub(r"(RTX|GTX)", r"GeForce \1", name)

  if "Core GPU" in name and not name.startswith("Apple"):
    name = "Apple " + name
  elif "Intel ARC" in name:
    name = "Intel Arc"
  elif "Iris" in name or "iris" in name:
    name = "Intel Iris Xe"
  elif name == "AMD Radeon 7":
    name = "AMD Radeon Vega 7"

  df.at[index, 'Graphics_name'] = name.strip()

In [93]:
for index, row in df.iterrows():
  name = row['Graphics_name']

  if name == 'AMD Vega':
    vega_mode = df[df['Graphics_name'].str.contains("Vega")]['Graphics_name'].mode()[0]
    name = vega_mode
  elif name == 'AMD':
    filtered = df[df['Graphics_name'].str.contains("AMD") & ~df['Graphics_name'].isin(["AMD"])]['Graphics_name']
    amd_mode = filtered.mode()[0]
    name = amd_mode
  elif name == 'Intel':
    filtered = df[df['Graphics_name'].str.contains("Intel") & ~df['Graphics_name'].isin(["Intel"])]['Graphics_name']
    int_mode = filtered.mode()[0]
    name = int_mode
  elif name == 'NVIDIA':
    filtered = df[df['Graphics_name'].str.contains("NVIDIA") & ~df['Graphics_name'].isin(["NVIDIA"])]['Graphics_name']
    nv_mode = filtered.mode()[0]
    name = nv_mode
  elif name == 'Intel UHD':
    filtered = df[df['Graphics_name'].str.contains("Intel") & ~df['Graphics_name'].isin(["Intel UHD"])]['Graphics_name']
    int_mode = filtered.mode()[0]
    name = int_mode

  df.at[index, 'Graphics_name'] = name.strip()

Перевіримо результат виправлення

In [94]:
sorted(df['Graphics_name'].unique())

['AMD Radeon',
 'AMD Radeon 610M',
 'AMD Radeon 680M',
 'AMD Radeon 780M',
 'AMD Radeon Pro 5500M',
 'AMD Radeon RX5600M',
 'AMD Radeon RX6500M',
 'AMD Radeon RX6550M',
 'AMD Radeon RX7600S',
 'AMD Radeon Vega 7',
 'AMD Radeon Vega 8',
 'AMD SoC',
 'ARM Mali G72',
 'Adreno 685',
 'Apple 10 Core GPU',
 'Apple 14 Core GPU',
 'Apple 18 Core GPU',
 'Apple 30 Core GPU',
 'Apple 40 Core GPU',
 'Apple 8 Core GPU',
 'Apple M1',
 'Arm Mali G52 MC2 2EE',
 'Arm Mali G72 MP3',
 'Intel Arc',
 'Intel Iris Xe',
 'Intel UHD 600',
 'Intel UHD 620',
 'NVIDIA GeForce GTX 1650',
 'NVIDIA GeForce GTX 1650 Max Q',
 'NVIDIA GeForce MX130',
 'NVIDIA GeForce MX450',
 'NVIDIA GeForce MX550',
 'NVIDIA GeForce MX570',
 'NVIDIA GeForce RTX 2050',
 'NVIDIA GeForce RTX 3000 Ada',
 'NVIDIA GeForce RTX 3050',
 'NVIDIA GeForce RTX 3050 Ti',
 'NVIDIA GeForce RTX 3060',
 'NVIDIA GeForce RTX 3070 Ti',
 'NVIDIA GeForce RTX 3080 Ti',
 'NVIDIA GeForce RTX 3500 Ada',
 'NVIDIA GeForce RTX 4050',
 'NVIDIA GeForce RTX 4060',
 'N

Висновок наступний: дані виправлено

Розглянемо характерстику "Operating_system"

In [95]:
sorted(df['Operating_system'].unique())

['Android 11 OS',
 'Chrome OS',
 'DOS 3.0 OS',
 'DOS OS',
 'Linux OS',
 'Mac 10.15.3\t OS',
 'Mac Catalina OS',
 'Mac OS',
 'Prime OS',
 'Ubuntu OS',
 'Windows 10 OS',
 'Windows 11 OS',
 'jio']

Операційні системи слід уніфікувати

In [96]:
for index, row in df.iterrows():
  name = row['Operating_system']
  if "Mac" in name:
    name = "Mac OS"
  if "jio" in name:
    name = "Jio OS"
  if "Ubuntu" in name:
    name = "Linux OS"
  if "DOS" in name:
    name = "DOS OS"
  df.at[index, 'Operating_system'] = name.strip()

Перевіримо правильність рішення проблеми

In [97]:
sorted(df['Operating_system'].unique())

['Android 11 OS',
 'Chrome OS',
 'DOS OS',
 'Jio OS',
 'Linux OS',
 'Mac OS',
 'Prime OS',
 'Windows 10 OS',
 'Windows 11 OS']

На цьому підготовка даних завершено. Збережемо дані у .csv файл

In [98]:
df.to_csv('Laptop_prices_clean.csv', index = False)
files.download("Laptop_prices_clean.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>